In [1]:
import pymongo
import numpy as np

#Connect to mongoDB and get data from collection
client = pymongo.MongoClient() 
db = client.TrainingDataDB 

currentExpData = []
for i in range(1,16):
    expCollectionName = "TakeOffExp"+str(i)
    
    expCollection = db[expCollectionName]
    
    expDocuments = expCollection.find({})
    
    for expDocument in expDocuments:
        expDataArray = expDocument['data']
        currentExpData.append(expDataArray)

#print currentExpData

expertDataArray = np.array(currentExpData)
print expertDataArray.shape


(2211L, 10L)


In [2]:
currentNovData = []      
for j in range (1,3):
    novCollectionName = "TakeOffNov"+str(j)
    
    novCollection = db[novCollectionName]
    
    novDocuments = novCollection.find({})
    
    for novDocument in novDocuments:
        novDataArray = novDocument['data']
        currentNovData.append(novDataArray)

#print currentNovData

noviceDataArray = np.array(currentNovData)
print noviceDataArray.shape

(148L, 10L)


In [3]:
#Consolidate all data 
allData = np.concatenate((expertDataArray,noviceDataArray), axis=0)
allData.shape

(2359L, 10L)

In [4]:
#Create labels
expert_rows, expert_columns = expertDataArray.shape
#print expert_rows

expert_labels = np.zeros(expert_rows,)
#print expert_labels

novice_rows, novice_columns = noviceDataArray.shape
#print novice_rows

novice_labels = np.ones(novice_rows,)
#print novice_labels

all_labels = np.concatenate((expert_labels,novice_labels), axis=0)
print all_labels

[ 0.  0.  0. ...,  1.  1.  1.]


In [5]:
from sklearn import preprocessing, cross_validation, neighbors, svm

Data_train, Data_test, Labels_train, Labels_test = cross_validation.train_test_split(allData, all_labels, test_size=0.2)
clf = svm.SVC(kernel='linear', C = 1.0)

In [6]:
#Fit the training data
clf.fit(Data_train, Labels_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
#Fit test data
clf.fit(Data_test, Labels_test)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
clf = svm.SVC(kernel='linear', C=1).fit(Data_train, Labels_train)

In [13]:
#Get results from cross validation
cross_val_score = clf.score(Data_test, Labels_test)
print cross_val_score


0.947033898305


In [16]:
#Output detailed results
#from sklearn.model_selection import cross_val_score
#clf = svm.SVC(kernel='linear', C=1)
#scores = cross_val_score(clf, allData, all_labels, cv=5)
#print scores

In [14]:
#Post cross validation score to DB

db = client.ResultsDataDB

name = "Cross Validation Score"
date = "Mar 11"
item = {
        "Name": name,
        "Result": result,
        "Date": date,
        "X-Val Score": cross_val_score
    }
result = db.ClassificationResults.insert_one(item)
result.inserted_id

ObjectId('58c4b05c352a601acc6e57c0')